## 02 - Exploratory Data Analysis (EDA) and Initial Cleaning

**Project:** UK Housing Price Paid Records

**Purpose:** To explore the data structure, confirm data quality, identify and justify handling of outliers (e.g., non-market sales), create necessary features (Sale\_Year), and generate visualizations and correlations to inform the final model.

**Team Member(s):** Tymo Verhaegen

**Cleaning Decisions Made Here:**
* Confirmed the absence of missing values.
* Filtered out all non-market transactions where `price <= £1`.
* Decided on renaming all columns to Python-friendly `snake_case`.

**Date Last Run:** 06/11/2025

---

In [12]:
import pandas as pd

# Define the path to your optimized Parquet file
parquet_file_path = '../data/housing/processed/price_paid_init.parquet'

print("Loading optimized data from Parquet...")

# Load the Parquet file. This should be very fast (seconds, not minutes!).
df = pd.read_parquet(parquet_file_path)

print(f"Data loaded successfully. Total records: {len(df):,}")
print("-" * 30)

# 1. Initial Inspection (Confirm types and check memory)
print("DataFrame Structure and Memory Usage:")
df.info(memory_usage='deep')

# 2. Check the first few rows
print("\nFirst 5 Rows:")
print(df.head())

Loading optimized data from Parquet...
Data loaded successfully. Total records: 22,489,348
------------------------------
DataFrame Structure and Memory Usage:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22489348 entries, 0 to 22489347
Data columns (total 11 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   Transaction unique identifier      string        
 1   Price                              int32         
 2   Date of Transfer                   datetime64[ns]
 3   Property Type                      category      
 4   Old/New                            category      
 5   Duration                           category      
 6   Town/City                          category      
 7   District                           category      
 8   County                             category      
 9   PPDCategory Type                   category      
 10  Record Status - monthly file only  category      
dtypes: ca

In [13]:
# --- Cleaning Step 1: Feature Engineering the Date ---

# Create a new column for the year of sale
df['Sale_Year'] = df['Date of Transfer'].dt.year

# Convert the new year column to a memory-efficient integer type
# Assuming we don't need any special features beyond the integer year
df['Sale_Year'] = df['Sale_Year'].astype('int16')

print("Created 'Sale_Year' column:")
print(df[['Date of Transfer', 'Sale_Year']].head())

Created 'Sale_Year' column:
  Date of Transfer  Sale_Year
0       1995-08-18       1995
1       1995-08-09       1995
2       1995-06-30       1995
3       1995-11-24       1995
4       1995-06-23       1995


In [14]:
# --- Cleaning Step 2: Standardizing Categorical Columns ---

# List all the columns that should be consistent
geo_cols = ['Town/City', 'District', 'County']

# We also check other object/category columns for consistency
cat_cols = ['Property Type', 'Old/New', 'Duration', 'PPDCategory Type', 'Record Status - monthly file only']

# Combine all columns that need string cleaning
columns_to_standardize = geo_cols + cat_cols

print("Starting string standardization for categorical/geographic columns...")

for col in columns_to_standardize:
    # Use str.title() to convert strings to Title Case (e.g., 'aBs' -> 'Abs')
    # .str.title() works well on CategoryDtype objects in pandas
    # .str.strip() removes any leading/trailing whitespace, which is a common issue in raw data
    df[col] = df[col].astype(str).str.strip().str.title().astype('category')
    
print("Casing standardized and extra whitespace removed.")

# Quick check on a column that should have clean values:
print(f"\nUnique values in 'Town/City' after cleaning (first 10):")
print(df['Town/City'].head(10).unique())

Starting string standardization for categorical/geographic columns...
Casing standardized and extra whitespace removed.

Unique values in 'Town/City' after cleaning (first 10):
['Oldham', 'Grays', 'Highbridge', 'Bedford', 'Wakefield', 'Salisbury', 'Witney', 'St. Austell', 'Greenford', 'Ferndale']
Categories (1170, object): ['Abbots Langley', 'Aberaeron', 'Aberdare', 'Aberdovey', ..., 'Yelverton', 'Yeovil', 'York', 'Ystrad Meurig']


In [15]:
# --- Cleaning Step 3: Handling Price Outliers (Zero/Non-Market Sales) ---

# The price paid data includes sales for £1 or £0. 
# These are non-market transfers (e.g., family gifts, internal company transfers).
# Keeping them will drag the average house price way down.

# 1. Store the count of these records for later reporting
count_low_price = len(df[df['Price'] <= 1])

# 2. Filter the main DataFrame to only include transactions with a price greater than £1.
# This is a common and necessary step for price prediction/analysis.
df_cleaned = df[df['Price'] > 1].copy()

# 3. Report the result
print("-" * 30)
print(f"Original Records: {len(df):,}")
print(f"Records removed (Price <= £1): {count_low_price:,}")
print(f"Cleaned Records Remaining: {len(df_cleaned):,}")

# Replace the original DataFrame 'df' with the cleaned one
df = df_cleaned
del df_cleaned # Clean up the temporary variable

------------------------------
Original Records: 22,489,348
Records removed (Price <= £1): 92
Cleaned Records Remaining: 22,489,256


In [16]:
# --- Cleaning Step 4: Check for Missing Values (Na-values) ---

print("\nMissing Value Check (Initial):")
print(df.isnull().sum())
print("-" * 30)


Missing Value Check (Initial):
Transaction unique identifier        0
Price                                0
Date of Transfer                     0
Property Type                        0
Old/New                              0
Duration                             0
Town/City                            0
District                             0
County                               0
PPDCategory Type                     0
Record Status - monthly file only    0
Sale_Year                            0
dtype: int64
------------------------------


In [17]:
# --- Cleaning Step 5: Rename Columns to be Python-Friendly ---

# Create a function to clean up column names
def clean_col_name(col):
    # Convert to lowercase, replace spaces with underscores, and strip any leading/trailing spaces
    return col.lower().replace(' ', '_').strip()

# Apply the function to all column names
df.columns = [clean_col_name(col) for col in df.columns]

print("\nColumns Renamed:")
print(df.columns.tolist())
print("-" * 30)

# Also rename the Date column to be more descriptive and remove the hyphen
df.rename(columns={'date_of_transfer': 'sale_date'}, inplace=True)

print("Final Columns:")
print(df.columns.tolist())


Columns Renamed:
['transaction_unique_identifier', 'price', 'date_of_transfer', 'property_type', 'old/new', 'duration', 'town/city', 'district', 'county', 'ppdcategory_type', 'record_status_-_monthly_file_only', 'sale_year']
------------------------------
Final Columns:
['transaction_unique_identifier', 'price', 'sale_date', 'property_type', 'old/new', 'duration', 'town/city', 'district', 'county', 'ppdcategory_type', 'record_status_-_monthly_file_only', 'sale_year']


In [18]:
# --- Cleaning Step 6: Categorical Feature Refinement ---
print("\n--- Cleaning Step 6: Categorical Feature Refinement ---")

# 6.1 Set the order for the 'duration' column (Leasehold vs Freehold)
# 'L' (Leasehold) is generally considered less valuable than 'F' (Freehold) for prediction
duration_mapping = {'L': 'Leasehold', 'F': 'Freehold'}
duration_order = ['Leasehold', 'Freehold']

df['duration'] = df['duration'].map(duration_mapping)
df['duration'] = pd.Categorical(df['duration'], categories=duration_order, ordered=True)
print("\n- Duration column updated to ordered category: ['Leasehold', 'Freehold'].")

# 6.2 Drop redundant columns
# 'ppd_category_type' ('A' is 99.9% of the data, 'B' is for additional transactions)
# 'record_status_-_monthly_file_only' ('A' for all records in the full file)
cols_to_drop = ['ppd_category_type', 'record_status_-_monthly_file_only']
df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

print(f"\n- Dropping redundant/constant columns:")
print(f"  - Dropped 'ppd_category_type' (99.9% value 'A').")
print(f"  - Dropped 'record_status_-_monthly_file_only' (100% value 'A').")

print(f"\nFinal number of columns: {len(df.columns)}")


--- Cleaning Step 6: Categorical Feature Refinement ---

- Duration column updated to ordered category: ['Leasehold', 'Freehold'].

- Dropping redundant/constant columns:
  - Dropped 'ppd_category_type' (99.9% value 'A').
  - Dropped 'record_status_-_monthly_file_only' (100% value 'A').

Final number of columns: 11
